#### Question 3 

#### Letter I Dataset with Circles

In this question, we have created datasets where the **Letter I** is formed using circles. 

- **Micro-object**: Circle  
- **Macro-object**: Letter I  

The objective of the model is not only to predict the **number of groups** correctly but also to determine whether the circles, when combined, form the **Letter I** or not. 

#### Model Performance
- Using a **basic CNN model**, the test accuracy achieved was **85%**.
- However, the model showed signs of **overfitting**. 

#### Model Architecture Modification
To address overfitting, we replaced the basic CNN with a **VGG architecture**. 

- With the **current number of epochs set to 5**, we achieved an accuracy of **82%**.  
- We aimed to train the model for a **higher number of epochs**, but each epoch took approximately **1 hour**, limiting our ability to run more iterations.
- We expect that running the **VGG model** for more epochs would likely increase the test accuracy.


In [28]:
##Final 1 -- dataset generation of I's

import numpy as np
import cv2
import os
import random

# Directory setup
train_dir = 'generalized_proximity/train'
test_dir = 'generalized_proximity/test'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define image size and object properties
image_size = (800, 800)  # Image size
object_radius = 10  # Radius of small shapes (circles)
object_color = (0, 0, 0)  # Color of objects
bg_color = (255, 255, 255)  # Background color

# Parameters for proximity grouping
group_spacing_far = 150  # Spacing between different groups

# Function to draw microobjects (only circles)
def draw_microobject(image, center_x, center_y):
    """Draw a small microobject like a circle."""
    cv2.circle(image, (center_x, center_y), object_radius, object_color, -1)
    return image

# Function to draw a macroobject (e.g., an 'I' made of circles)
def draw_macroobject(image, start_x, start_y, num_microobjects=8):
    """Draw a macroobject formed by multiple microobjects."""
    for i in range(num_microobjects):
        center_y = start_y + i * (2 * object_radius + 5)  # Adjust vertical spacing
        image = draw_microobject(image, start_x, center_y)
    return image

# Function to draw proximity groups of macroobjects
def draw_proximity_groups(image, num_groups=1):
    """Draw proximity groups, each containing varying numbers of macroobjects."""
    valid_I_groups = 0  # Track the number of valid "I" groups
    for group in range(num_groups):
        group_start_x = 50 + group * group_spacing_far  # Start position of each group
        num_macroobjects = random.randint(1, 4)  # Random number of I's in this group
        
        # Check if the group can be considered an "I"
        if num_macroobjects <= 3:
            valid_I_groups += 1  # Increment valid group count
            
        for i in range(num_macroobjects):
            start_x = group_start_x + i * (3 * object_radius + 5)  # Adjust horizontal spacing
            image = draw_macroobject(image, start_x, 50)
    
    return image, valid_I_groups  # Return the valid count

# Function to generate images with random numbers of groups and varying macroobjects
def generate_image(image_num, output_dir, num_groups):
    """Generates an image with a specified number of groups, each with varying numbers of macroobjects."""
    img = np.ones((image_size[1], image_size[0], 3), dtype=np.uint8) * 255  # Blank white image
    img, valid_I_groups = draw_proximity_groups(img, num_groups=num_groups)

    # Save the image with the number of valid "I" groups as the filename
    # img_name = f"{valid_I_groups}.png"
    img_name = f"{valid_I_groups}_{image_num}.png"
    img_path = os.path.join(output_dir, img_name)
    cv2.imwrite(img_path, img)

# Generate train and test datasets
def generate_dataset(num_train, num_test, max_groups=3):
    """Generates train and test datasets with varying numbers of groups and varying numbers of macroobjects."""
    for i in range(num_train):
        num_groups = random.randint(1, max_groups)  # Random number of groups
        generate_image(i, train_dir, num_groups=num_groups)
    
    for i in range(num_test):
        num_groups = random.randint(1, max_groups)  # Random number of groups
        generate_image(i, test_dir, num_groups=num_groups)

# Define number of images for train and test
num_train = 800  # Training images
num_test = 100    # Testing images
max_groups = 5   # Maximum number of groups in an image

# Generate the datasets
generate_dataset(num_train, num_test, max_groups=max_groups)

print(f"Training and testing datasets generated with {num_train} training and {num_test} testing images.")

Training and testing datasets generated with 800 training and 100 testing images.


In [32]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models

# Directory setup
train_dir = 'generalized_proximity/train'
test_dir = 'generalized_proximity/test'

# Parameters
image_size = (800, 800)  # Image dimensions
num_classes = 6          # Adjust this based on your dataset

# Function to load dataset
def load_data(data_dir):
    images = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".png"):
            # Load and resize image
            img_path = os.path.join(data_dir, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, image_size)
            images.append(img)
            
            # Extract label from filename (valid I groups are the first part of the filename)
            label = int(filename.split('_')[0])  # Get the number of valid "I" groups
            labels.append(label)
    return np.array(images), np.array(labels)

# Load the datasets
X_train, y_train = load_data(train_dir)
X_test, y_test = load_data(test_dir)

# Normalize the images
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Check unique labels to understand the range of your dataset
print("Unique labels in training set:", np.unique(y_train))
print("Unique labels in testing set:", np.unique(y_test))

# Validate label range
if np.any(y_train >= num_classes) or np.any(y_test >= num_classes):
    print("Error: Found labels out of range! Adjusting num_classes accordingly.")

# Convert labels to categorical format
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Define a simple CNN model
def create_simple_cnn_model():
    model = models.Sequential()
    
    # Block 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(image_size[0], image_size[1], 3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Block 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Block 3
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten the output and add fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization
    model.add(layers.Dense(num_classes, activation='softmax'))  # Output layer for classification
    
    return model

# Compile the model
model = create_simple_cnn_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

# Save the model
model.save('simple_cnn_model.h5')

Unique labels in training set: [0 1 2 3 4 5]
Unique labels in testing set: [0 1 2 3 4 5]
Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 359s 17s/step - accuracy: 0.2208 - loss: 23.6576 - val_accuracy: 0.1187 - val_loss: 4.8017
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 325s 16s/step - accuracy: 0.5461 - loss: 1.0995 - val_accuracy: 0.1063 - val_loss: 12.5685
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 328s 16s/step - accuracy: 0.7083 - loss: 0.7220 - val_accuracy: 0.1437 - val_loss: 16.0620
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 326s 16s/step - accuracy: 0.8256 - loss: 0.4408 - val_accuracy: 0.1437 - val_loss: 30.4438
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 287s 14s/step - accuracy: 0.8554 - loss: 0.3728 - val_accuracy: 0.1437 - val_loss: 39.3559
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 270s 13s/step - accuracy: 0.9010 - loss: 0.2523 - val_accuracy: 0.1437 - val_loss: 41.7952
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 335s 17s/step - accuracy: 0.9082 - loss: 0.2547 - val_accuracy: 0.1437 - val_loss: 55.4583
Epoch 8/

Test accuracy: 0.8500


In [34]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models

# Directory setup
train_dir = 'generalized_proximity/train'
test_dir = 'generalized_proximity/test'

# Parameters
image_size = (800, 400)  
num_classes = 6         

# Function to load dataset
def load_data(data_dir):
    images = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".png"):
            # Load and resize image
            img_path = os.path.join(data_dir, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, image_size)
            images.append(img)
            
            # Extract label from filename (valid I groups are the first part of the filename)
            label = int(filename.split('_')[0])  # Get the number of valid "I" groups
            labels.append(label)
    return np.array(images), np.array(labels)

# Load the datasets
X_train, y_train = load_data(train_dir)
X_test, y_test = load_data(test_dir)

# Normalize the images
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Check unique labels to understand the range of your dataset
print("Unique labels in training set:", np.unique(y_train))
print("Unique labels in testing set:", np.unique(y_test))

# Convert labels to categorical format
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Define a VGG-like CNN model
def create_vgg_model():
    model = models.Sequential()
    
    # Block 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(image_size[0], image_size[1], 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Block 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Block 3
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Block 4
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization
    model.add(layers.Dense(num_classes, activation='softmax'))  # Output layer for classification
    
    return model

# Compile the model
model = create_vgg_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=1, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

# Save the model
model.save('proximity_vgg_model.h5')

Unique labels in training set: [0 1 2 3 4 5]
Unique labels in testing set: [0 1 2 3 4 5]
Epoch 1/5
640/640 ━━━━━━━━━━━━━━━━━━━━ 4077s 6s/step - accuracy: 0.4460 - loss: 740.0128 - val_accuracy: 0.0000e+00 - val_loss: 7605.6665
Epoch 2/5
640/640 ━━━━━━━━━━━━━━━━━━━━ 4018s 6s/step - accuracy: 0.7252 - loss: 217.5879 - val_accuracy: 0.1375 - val_loss: 6736.2476
Epoch 3/5
640/640 ━━━━━━━━━━━━━━━━━━━━ 4000s 6s/step - accuracy: 0.8274 - loss: 72.2046 - val_accuracy: 0.1437 - val_loss: 7400.1108
Epoch 4/5
640/640 ━━━━━━━━━━━━━━━━━━━━ 4020s 6s/step - accuracy: 0.8854 - loss: 48.8125 - val_accuracy: 0.1187 - val_loss: 7547.6421
Epoch 5/5
640/640 ━━━━━━━━━━━━━━━━━━━━ 4023s 6s/step - accuracy: 0.9016 - loss: 32.8328 - val_accuracy: 0.1312 - val_loss: 8395.6230
4/4 ━━━━━━━━━━━━━━━━━━━━ 55s 9s/step - accuracy: 0.8801 - loss: 823.1250


Test accuracy: 0.8200


Acknowledgment:  
    This assignment is collaboratively done by:   
    Keerthana - 210290  
    Meghana - 210073  
    Madhuri - 210568  
    Shobhit Sharma - 210992